<a href="https://colab.research.google.com/github/mehakhafeez/lab-chatbot-with-multi-query-retriever/blob/main/lab-chatbot-with-multi-query-retriever.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question Answering with LangChain, OpenAI, and MultiQuery Retriever

This interactive workbook demonstrates example of Elasticsearch's [MultiQuery Retriever](https://api.python.langchain.com/en/latest/retrievers/langchain.retrievers.multi_query.MultiQueryRetriever.html) to generate similar queries for a given user input and apply all queries to retrieve a larger set of relevant documents from a vectorstore.

Before we begin, we first split the fictional workplace documents into passages with `langchain` and uses OpenAI to transform these passages into embeddings and then store these into Elasticsearch.

We will then ask a question, generate similar questions using langchain and OpenAI, retrieve relevant passages from the vector store, and use langchain and OpenAI again to provide a summary for the questions.

## Install packages and import modules

In [1]:
!pip show langchain-elasticsearch

In [2]:
!python3 -m pip install -qU jq lark langchain langchain-elasticsearch langchain_openai tiktoken

from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_elasticsearch import ElasticsearchStore
from langchain_openai.llms import OpenAI
from langchain.retrievers.multi_query import MultiQueryRetriever
from getpass import getpass

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.0/66.0 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.6/746.6 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.0/111.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.6/45.6 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.2/54.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.2/571.2 kB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.5/64.5 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 632.7/632.7 kB 10.8 MB/s eta 0:00:00


## Connect to Elasticsearch

ℹ️ We're using an Elastic Cloud deployment of Elasticsearch for this notebook. If you don't have an Elastic Cloud deployment, sign up [here](https://cloud.elastic.co/registration?utm_source=github&utm_content=elasticsearch-labs-notebook) for a free trial.

We'll use the **Cloud ID** to identify our deployment, because we are using Elastic Cloud deployment. To find the Cloud ID for your deployment, go to https://cloud.elastic.co/deployments and select your deployment.

We will use [ElasticsearchStore](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html) to connect to our elastic cloud deployment, This would help create and index data easily.  We would also send list of documents that we created in the previous step

In [3]:
# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#finding-your-cloud-id
ELASTIC_CLOUD_ID = getpass("Elastic Cloud ID: ")

# https://www.elastic.co/search-labs/tutorials/install-elasticsearch/elastic-cloud#creating-an-api-key
ELASTIC_API_KEY = getpass("Elastic Api Key: ")

# https://platform.openai.com/api-keys
OPENAI_API_KEY = getpass("OpenAI API key: ")

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)

vectorstore = ElasticsearchStore(
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
    index_name="nasa_sales_team_documents", #give it a meaningful name,
    embedding=embeddings,
)

Elastic Cloud ID: ··········
Elastic Api Key: ··········
OpenAI API key: ··········


## Indexing Data into Elasticsearch
Let's download the sample dataset and deserialize the document.

In [4]:
from urllib.request import urlopen
import json

url = "https://raw.githubusercontent.com/elastic/elasticsearch-labs/main/example-apps/chatbot-rag-app/data/data.json"

response = urlopen(url)
data = json.load(response)

with open("temp.json", "w") as json_file:
    json.dump(data, json_file)

### Split Documents into Passages

We’ll chunk documents into passages in order to improve the retrieval specificity and to ensure that we can provide multiple passages within the context window of the final question answering prompt.

Here we are chunking documents into 800 token passages with an overlap of 400 tokens.

Here we are using a simple splitter but Langchain offers more advanced splitters to reduce the chance of context being lost.

In [7]:
!pip install -U langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 76.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.4 MB/s eta 0:00:00


In [16]:
from langchain_elasticsearch._sync.vectorstores import ElasticsearchStore
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import JSONLoader
from getpass import getpass
import json
from urllib.request import urlopen

def metadata_func(record: dict, metadata: dict) -> dict:
    metadata['name'] = record.get('name', 'Unknown')
    metadata['summary'] = record.get('summary', 'No summary available.')
    metadata['url'] = record.get('url', 'No URL available.')
    metadata['category'] = record.get('category', 'No category provided.')

    return metadata

# For more loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/
# And 3rd party loaders https://python.langchain.com/docs/modules/data_connection/document_loaders/#third-party-loaders
loader = JSONLoader(
    file_path="temp.json",
    jq_schema=".[]",
    content_key="content",
    metadata_func=metadata_func,
)

text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=800,  # Example chunk size of 800 tokens
    chunk_overlap=400 # Example chunk overlap of 400 tokens
)
docs = loader.load_and_split(text_splitter=text_splitter)

In [18]:
# Ensure the updated_at field is a valid ISO 8601 date or None
def ensure_updated_at_format(record: dict, metadata: dict) -> dict:
    updated_at_raw = record.get('updated_at', None)
    try:
        metadata['updated_at'] = (
            datetime.fromisoformat(updated_at_raw).isoformat()
            if updated_at_raw else None
        )
    except (ValueError, TypeError):
        metadata['updated_at'] = None  # Use None if the value isn't valid

    return metadata

### Bulk Import Passages

Now that we have split each document into the chunk size of 800, we will now index data to elasticsearch using [ElasticsearchStore.from_documents](https://api.python.langchain.com/en/latest/vectorstores/langchain.vectorstores.elasticsearch.ElasticsearchStore.html#langchain.vectorstores.elasticsearch.ElasticsearchStore.from_documents).

We will use Cloud ID, Password and Index name values set in the `Create cloud deployment` step.

In [19]:
documents = vectorstore.from_documents(
    docs,
    embeddings,
    index_name="nasa_sales_team_documents",
    es_cloud_id=ELASTIC_CLOUD_ID,
    es_api_key=ELASTIC_API_KEY,
)

llm = OpenAI(temperature=0, openai_api_key=OPENAI_API_KEY)

retriever = MultiQueryRetriever.from_llm(vectorstore.as_retriever(), llm)

# Question Answering with MultiQuery Retriever

Now that we have the passages stored in Elasticsearch, we can now ask a question to get the relevant passages.

In [20]:
from langchain.schema.runnable import RunnableParallel, RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.schema import format_document

import logging

logging.basicConfig()
logging.getLogger("langchain.retrievers.multi_query").setLevel(logging.INFO)

LLM_CONTEXT_PROMPT = ChatPromptTemplate.from_template(
    """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Be as verbose and educational in your response as possible.

    context: {context}
    Question: "{question}"
    Answer:
    """
)

LLM_DOCUMENT_PROMPT = PromptTemplate.from_template(
    """
---
SOURCE: {name}
{page_content}
---
"""
)


def _combine_documents(
    docs, document_prompt=LLM_DOCUMENT_PROMPT, document_separator="\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)


_context = RunnableParallel(
    context=retriever | _combine_documents,
    question=RunnablePassthrough(),
)

chain = _context | LLM_CONTEXT_PROMPT | llm

ans = chain.invoke("what is the nasa sales team?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide information on the sales team at NASA?', '2. How does the sales team operate within NASA?', '3. What are the responsibilities of the NASA sales team?']


---- Answer ----
The NASA sales team is a part of the Americas region in the sales organization of the company. It is led by two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. The team is responsible for promoting and selling the company's products and services to potential clients in the North America and South America regions. They work closely with other departments, such as marketing, product development, and customer support, to ensure the company consistently delivers high-quality products and services to its clients.


**Generate at least two new iteratioins of the previous cells - Be creative.** Did you master Multi-
Query Retriever concepts through this lab?

In [21]:
ans = chain.invoke("Can you tell me about the sales department at NASA?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What information can you provide about the sales department at NASA?', '2. Could you give me some insights on the sales department at NASA?', "3. I'm interested in learning more about the sales department at NASA, can you assist me?"]


---- Answer ----

NASA does not have a sales department as it is a government agency focused on space exploration and research. However, the context provided mentions a region called North America South America (NASA) which has two Area Vice-Presidents, Laura Martinez for North America and Gary Johnson for South America. This region is part of the sales organization of a tech company and is responsible for promoting and selling the company's products and services in the United States, Canada, Mexico, Central and South America.


In [22]:
ans = chain.invoke("What specific responsibilities do the individual members of the NASA sales team have?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. Can you provide a breakdown of the roles and responsibilities of each member in the NASA sales team?', '2. How are the responsibilities divided among the individual members of the NASA sales team?', '3. What are the specific tasks and duties assigned to each member of the NASA sales team?']


---- Answer ----

The NASA sales team consists of two Area Vice-Presidents, Laura Martinez and Gary Johnson. As Area Vice-Presidents, they are responsible for overseeing the sales operations in North America and South America, respectively. This includes identifying and pursuing new business opportunities, nurturing existing client relationships, and ensuring customer satisfaction. They also collaborate closely with other departments, such as marketing, product development, and customer support, to ensure the company consistently delivers high-quality products and services to its clients. Additionally, they lead a team of dedicated account managers, sales representatives, and support staff in their respective regions.


In [23]:
ans = chain.invoke("How does the NASA sales team collaborate with other departments?")

print("---- Answer ----")
print(ans)

INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the collaboration process between the NASA sales team and other departments?', '2. Can you explain the methods used by the NASA sales team to work with other departments?', '3. How do different departments within NASA work together to achieve sales goals?']


---- Answer ----
The NASA sales team collaborates with other departments, such as marketing, product development, and customer support, to ensure the consistent delivery of high-quality products and services to clients. This collaboration involves sharing information, coordinating efforts, and working together to achieve the company's goals and objectives. For example, the sales team may provide feedback on customer needs and market trends to the product development team, while the marketing team may support the sales team with targeted campaigns and materials. This collaboration helps to ensure that the company's products and services meet customer needs and expectations, leading to increased revenue and customer satisfaction.


In [24]:
ans = chain.invoke("How is the sales team divided regionally within NASA?")

print("---- Answer ----")
print(ans)


INFO:langchain.retrievers.multi_query:Generated queries: ['1. What is the regional breakdown of the sales team at NASA?', '2. Can you provide information on the geographical distribution of the sales team at NASA?', "3. How does NASA's sales team divide their operations across different regions?"]


---- Answer ----
The sales team within NASA is divided into two main regions: North America and South America. Each region has its own Area Vice-President, with Laura Martinez leading North America and Gary Johnson leading South America.
